In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv
import polars as pl

dotenv_path = os.path.abspath("../.env")  # Adjust the path accordingly
load_dotenv(dotenv_path)

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "fairusecases")

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API"]

# Initialize the LLM (ensure your API key is set in your environment)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    max_tokens=None,
    timeout=None
)

fair_use_summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a legal analyst. Read the following legal opinion and summarize its significance with respect to its Fair Use ruling. Focus on the key points, include a general overview of the facts of the case and the context, as well as the implications regarding Fair Use.\n\n""",
        ),
        ("human", "Here is the legal opinion: {legal_opinion}"),
    ]
)

fair_use_summary_chain = fair_use_summary_prompt | llm

/Users/jho/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_opinions(tx):
    
    results = tx.run("""
        MATCH (o:Opinion)
        WHERE o.Summary IS NULL
        RETURN o.Document as Document
        """
    )
         
    return pl.from_pandas(results.to_df())


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
            df = session.execute_read(get_opinions)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: Summary)} {position: line: 3, column: 17, offset: 43} for query: '\n        MATCH (o:Opinion)\n        WHERE o.Summary IS NULL\n        RETURN o.Document as Document\n        '


In [3]:
df.describe

<bound method DataFrame.describe of shape: (283, 1)
┌─────────────────────────────┐
│ Document                    │
│ ---                         │
│ str                         │
╞═════════════════════════════╡
│ <div>                       │
│ <center><b><span class="ci… │
│ <div>                       │
│ <center><b><span class="ci… │
│ <opinion type="majority">   │
│ <p id=…                     │
│                             │
│ --- Page 1 ---              │
│ Page 301                    │
│ 960 F.2…                    │
│ …                           │
│                             │
│ --- Page 1 ---              │
│ 464 F.Supp.3d 59…           │
│                             │
│ --- Page 1 ---              │
│ ESTATE OF JAMES …           │
│                             │
│ --- Page 1 ---              │
│ Colby, Alexandra…           │
│                             │
│ --- Page 1 ---              │
│ Colby, Alexandra…           │
└─────────────────────────────┘>

In [ ]:
def create_opinion_summary(tx, document, summary):
    
    tx.run("""
        MATCH (o:Opinion {Document: $document})
        SET o.Summary = $summary
        """, document = document, summary = summary
    )
         

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
            for d in df.iter_rows(named = True):
                summary = fair_use_summary_chain.invoke({"legal_opinion": d["Document"]}).content
                session.execute_write(create_opinion_summary, d["Document"], summary)